In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier 
from sklearn.impute import SimpleImputer

In [7]:
df = pd.read_csv("D:\\ml project\\data\\EDA_aviation_data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72889 entries, 0 to 72888
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0.1            72889 non-null  int64  
 1   Unnamed: 0              72889 non-null  int64  
 2   event_id                72889 non-null  object 
 3   investigation_type      72889 non-null  object 
 4   event_date              72889 non-null  object 
 5   location                72860 non-null  object 
 6   country                 72677 non-null  object 
 7   injury_severity         72889 non-null  object 
 8   aircraft_damage         70773 non-null  object 
 9   aircraft_category       72889 non-null  object 
 10  make                    72889 non-null  object 
 11  model                   72889 non-null  object 
 12  amateur_built           72888 non-null  object 
 13  number_of_engines       68973 non-null  float64
 14  engine_type             68359 non-null

C:\Users\garvg\AppData\Local\Temp\ipykernel_21568\2829610289.py:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("D:\\ml project\\data\\EDA_aviation_data.csv")


In [8]:
df = df.drop(columns = ["Unnamed: 0","event_id" ,"model","location"],errors = "ignore")
df.columns.tolist()

['Unnamed: 0.1',
 'investigation_type',
 'event_date',
 'country',
 'injury_severity',
 'aircraft_damage',
 'aircraft_category',
 'make',
 'amateur_built',
 'number_of_engines',
 'engine_type',
 'purpose_of_flight',
 'total_fatal_injuries',
 'total_serious_injuries',
 'total_minor_injuries',
 'total_uninjured',
 'weather_condition',
 'broad_phase_of_flight']

In [9]:
df["event_date"] = pd.to_datetime(df["event_date"],errors = "coerce")
df["year"] = df["event_date"].dt.year
df["month"] =  df["event_date"].dt.month
df["day"] = df["event_date"].dt.day

In [10]:
df = df.drop(columns = ["event_date"])

In [11]:
df["target"]= df["injury_severity"].map({"fatal":1,"non-fatal":0}) 
df = df.drop(columns = ["injury_severity"])

In [12]:
low_cardinality = [
    "investigation_type",
    "aircraft_damage",
    "aircraft_category",
    "amateur_built",
    "purpose_of_flight",
    "weather_condition",
    "broad_phase_of_flight",
    "engine_type"
]

high_cardinality = ["make", "country"]

# NUMERIC
numeric_features = [
    "number_of_engines",
    "year", "month", "day"
]

In [13]:
cols_to_drop = [
    "total_fatal_injuries",
    "total_serious_injuries",
    "total_minor_injuries",
    "total_uninjured"
]
df = df.drop(columns=cols_to_drop, errors="ignore")

In [14]:
x = df[low_cardinality + high_cardinality + numeric_features]
y = df["target"]

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2 , random_state = 42 ,stratify = y)

In [15]:
preprocess = ColumnTransformer(
    transformers=[
        ("onehot", Pipeline([
            ("impute", SimpleImputer(strategy="most_frequent")),
            ("encode", OneHotEncoder(handle_unknown="ignore"))
        ]), low_cardinality),

        ("target", Pipeline([
            ("impute", SimpleImputer(strategy="most_frequent")),
            ("encode", TargetEncoder())
        ]), high_cardinality),

        ("numeric", Pipeline([
            ("impute", SimpleImputer(strategy="mean"))
        ]), numeric_features)
    ]
)

In [16]:
df.to_csv("D:\\ml project\\data\\final_aviation_data.csv", index=False)

In [17]:
model = Pipeline(steps=[
    ("preprocess", preprocess),
    ("classifier", RandomForestClassifier(
        n_estimators=300,
        max_depth=15,
        random_state=42
    ))
])

In [18]:
model.fit(x_train,y_train)

,steps,"[('preprocess', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('onehot', ...), ('target', ...), ...]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [19]:
from sklearn.metrics import classification_report

y_pred = model.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.95      0.93     11642
           1       0.76      0.63      0.69      2936

    accuracy                           0.88     14578
   macro avg       0.83      0.79      0.81     14578
weighted avg       0.88      0.88      0.88     14578



## TRAINING XGboost 

In [20]:
!pip install xgboost

In [21]:
from xgboost import XGBClassifier

In [22]:
xgb = XGBClassifier(n_estimators = 500, max_depth = 6, learning_rate = 0.05,subsample = 0.8,colsample_bytree = 0.8,scale_pos_weight = (len(y_train)/y_train.sum()),eval_metrix = "logloss",random_state = 42)
model = Pipeline(steps = [("prepocessor",preprocess),("classifier",xgb)])


In [23]:
model.fit(x_train,y_train)
preds  = model.predict(x_test)
print(classification_report(y_test,preds))

c:\Users\garvg\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:199: UserWarning: [23:43:06] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "eval_metrix" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


              precision    recall  f1-score   support

           0       0.95      0.87      0.91     11642
           1       0.62      0.81      0.70      2936

    accuracy                           0.86     14578
   macro avg       0.78      0.84      0.80     14578
weighted avg       0.88      0.86      0.87     14578



## HYPERTUNING PARAMETERS

In [31]:
param_distributions = {
    "classifier__n_estimators" : [200,400,600,800],
    "classifier__max_depth" : [3,4,5,6,7],
    "classifier__learning_rate" : [0.01,0.03,0.05,0.1],
    "classifier__subsample" : [0.6,0.8,1.0],
    "classifier__colsample_bytree" : [0.6,0.8,1.0],
    "classifier__min_child_weight" : [1,3,9,7],
    "classifier__gamma": [1,3,5,7],
    "classifier__scale_pos_weight" : [1,2,3,4]
}

random_search = RandomizedSearchCV(
    estimator = model,
    param_distributions=param_distributions,
    n_iter=20,                     # start with 15-20; increase if you have time
    scoring="f1",                  # f1 for positive class (fatal = 1)
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

random_search.fit(x_train,y_train)


Fitting 3 folds for each of 20 candidates, totalling 60 fits


c:\Users\garvg\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:199: UserWarning: [23:57:30] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "eval_metrix" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


,estimator,"Pipeline(step...=None, ...))])"
,param_distributions,"{'classifier__colsample_bytree': [0.6, 0.8, ...], 'classifier__gamma': [1, 3, ...], 'classifier__learning_rate': [0.01, 0.03, ...], 'classifier__max_depth': [3, 4, ...], ...}"
,n_iter,20
,scoring,'f1'
,n_jobs,-1
,refit,True
,cv,3
,verbose,2
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


In [33]:
print("Best F1 (cv):", random_search.best_score_)
print("Best Params:")
for k, v in random_search.best_params_.items():
    print(f"  {k}: {v}")

# 6. Evaluate best model on test set
best_model = random_search.best_estimator_

y_pred = best_model.predict(x_test)
print("\nClassification report on TEST set:")
print(classification_report(y_test, y_pred))

Best F1 (cv): 0.7109865596499906
Best Params:
  classifier__subsample: 0.8
  classifier__scale_pos_weight: 2
  classifier__n_estimators: 200
  classifier__min_child_weight: 7
  classifier__max_depth: 6
  classifier__learning_rate: 0.1
  classifier__gamma: 5
  classifier__colsample_bytree: 0.6

Classification report on TEST set:
              precision    recall  f1-score   support

           0       0.94      0.92      0.93     11642
           1       0.69      0.76      0.72      2936

    accuracy                           0.88     14578
   macro avg       0.82      0.84      0.83     14578
weighted avg       0.89      0.88      0.89     14578



In [ ]:
import joblib
joblib.dump(best_model, 'aviation_severity_model.pkl')

['aviation_incident_model.pkl']